In [ ]:
%%javascript
$('#run_all_cells_below').click()

In [ ]:
from hypothesis import settings

settings.register_profile("presentation", settings(
    database_file=None, 
    max_examples=100, 
    stateful_step_count=1000,
))

settings.load_profile("presentation")

# Property Based Testing
## Using Hypothesis<br><br><br>
### Amsterdam Python Meetup
### 26 April 2017<br><br><br>
### Daniel Bradburn

Property based testing

Choosing properties

Generating data

Stateful testing

## Property Based Testing

In [ ]:
def encode(text):
    """
    Run length encode text - repeated characters are replaced
    by a single instance followed by the count.
    """
    prev, count, result = '', '', []
    
    for curr in text:
        if prev == curr:
            count += 1
        else:
            result.append(f'{prev}{count}')
            prev, count = curr, 1
    else:
        result.append(f'{curr}{count}')
        
    return ''.join(result)

In [ ]:
encode('hello')

In [ ]:
from re import sub

def decode(text):
    """
    Run length decode text - characters followed by a count n
    are replaced by the character repeated n times.
    """
    grouper = lambda m: m.group(1) * int(m.group(2))
    return sub(r'(\D)(\d+)', grouper, text)

In [ ]:
decode('h1e1l2o1')

In [ ]:
def test_run_length_encode():
    assert encode('hello') == 'h1e1l2o1'

In [ ]:
!sh pytest.sh test_run_length_encode

In [ ]:
def test_run_length_decode():
    assert decode('h1e1l2o1') == 'hello'

In [ ]:
!sh pytest.sh test_run_length_decode

In [ ]:
import pytest

examples = ['hello', 'python', 'meetup', 'amsterdam']

@pytest.mark.parametrize('text', examples)
def test_parameterized_run_length_encode_decode(text):
    assert decode(encode(text)) == text

In [ ]:
!sh pytest.sh test_parameterized_run_length_encode_decode

In [ ]:
from random import seed, choice, randint

seed(0)

randletter = lambda _: chr(choice(range(32, 255)))
randrange  = lambda length: range(randint(0, length))
randchars  = lambda max_len: map(randletter, randrange(max_len))
randword   = lambda max_len: ''.join(randchars(max_len))
randwords  = lambda n, max_len: (randword(max_len) for _ in range(n))

@pytest.mark.parametrize('text', randwords(n=5, max_len=5))
def test_fuzzed_run_length_encode_decode(text):
    assert decode(encode(text)) == text

In [ ]:
!sh pytest.sh test_fuzzed_run_length_encode_decode

In [ ]:
from hypothesis import strategies as st, given

@given(st.text())
def check_property_rle_encode_decode_should_round_trip(text):
    assert decode(encode(text)) == text

In [ ]:
!sh pytest.sh check_property_rle_encode_decode_should_round_trip

In [ ]:
def encode_fixed(text):
    """
    Run length encode text - repeated characters are replaced
    by a single instance followed by the count.
    """
    curr, prev, count, result = '', '', '', []
    
    for curr in text:
        if prev == curr:
            count += 1
        else:
            result.append(f'{prev}{count}')
            prev, count = curr, 1
    else:
        result.append(f'{curr}{count}')
        
    return ''.join(result)

In [ ]:
@given(st.text())
def check_property_fixed_rle_encode_decode_should_round_trip(text):
    assert decode(encode_fixed(text)) == text

In [ ]:
!sh pytest.sh check_property_fixed_rle_encode_decode_should_round_trip

In [ ]:
from string import digits

@given(st.text(st.characters(blacklist_characters=digits)))
def check_property_nonum_rle_encode_decode_should_round_trip(text):
    assert decode(encode_fixed(text)) == text

In [ ]:
!sh pytest.sh check_property_nonum_rle_encode_decode_should_round_trip

In [ ]:
from hypothesis import settings, Verbosity

@settings(verbosity=Verbosity.verbose)
@given(st.text())
def check_property_verbose_rle_encode_decode_should_round_trip(text):
    assert decode(encode_fixed(text)) == text

In [ ]:
!sh pytest.sh check_property_verbose_rle_encode_decode_should_round_trip

In [ ]:
from random import seed

seed(0)

@pytest.mark.parametrize('text', randwords(n=5, max_len=15))
def test_fuzzed_more_run_length_encode_decode(text):
    assert decode(encode_fixed(text)) == text

In [ ]:
!sh pytest.sh test_fuzzed_more_run_length_encode_decode

&nbsp;

* Manual Testing

* Example Based Testing

*  Parameterized Tests

*  Property Based Tests

## Property patterns

In [ ]:
@given(st.text())
def check_property_dont_explode(text):
    text.encode('ascii')

In [ ]:
!sh pytest.sh check_property_dont_explode

In [ ]:
@given(st.text())
def check_property_there_and_back_again(text):
    assert text.encode('utf-8').decode('utf-8') == text

In [ ]:
!sh pytest.sh check_property_there_and_back_again

In [ ]:
@given(st.lists(st.integers(), min_size=1))
def check_property_round_and_around(c):
    assert c[::-1][::-1] == c

In [ ]:
!sh pytest.sh check_property_round_and_around

In [ ]:
@given(st.integers(), st.integers())
def check_property_different_paths_same_destination(x, y):
    assert x + y == y + x

In [ ]:
!sh pytest.sh check_property_different_paths_same_destination

In [ ]:
from heapq import heapify, heappop

@given(st.lists(st.integers(), min_size=1))
def check_property_some_things_never_change(c):
    smallest = min(c)
    heapify(c)
    assert heappop(c) == smallest

In [ ]:
!sh pytest.sh check_property_some_things_never_change

In [ ]:
@given(st.lists(st.integers()))
def check_property_the_more_things_change_the_more_they_stay_the_same(c):
    assert set(c) == set(set(c))

In [ ]:
!sh pytest.sh check_property_the_more_things_change_the_more_they_stay_the_same

In [ ]:
from dataset import connect

@given(st.lists(st.integers(min_value=0, max_value=1e6), min_size=1))
def check_property_two_heads_are_better_than_one(numbers):
    
    db = connect('sqlite:///:memory:')
    db['nums'].insert_many({'num': x} for x in numbers)
    
    actual = next(db.query('select sum(num) s from nums'))['s']
    expected = sum(numbers)
   
    assert actual == expected

In [ ]:
!sh pytest.sh check_property_two_heads_are_better_than_one

&nbsp;

* No errors

* Function pairs

* Commutivity

* Inverse

* Invariants

* Idempotency

* Model

## Data generation

In [ ]:
from re import findall
from stdnum.isin import is_valid

def extract_isin_codes(text):
    """
    Extract strings conforming to the isin code standard
    from words in text.
    """
    regex = '[a-zA-Z]{2}[a-zA-Z0-9]{9}[0-9]'
    possible_isins = findall(regex, text)
    return filter(is_valid, possible_isins)

In [ ]:
from hypothesis import given, strategies as st

@settings(verbosity=Verbosity.verbose)
@given(st.text())
def check_property_extract_isin_codes_1(text):
    extract_isin_codes(text)

In [ ]:
!sh pytest.sh check_property_extract_isin_codes_1 -s

In [ ]:
from string import ascii_letters, digits
from stdnum.isin import _country_codes, from_natid

@st.composite
def st_isin_codes(draw):
    country_code = draw(st.sampled_from(_country_codes))
    alphanum = ascii_letters + digits
    natid = draw(st.text(alphanum, min_size=9, max_size=9))
    return from_natid(country_code, natid)

In [ ]:
st_isin_codes().example()

In [ ]:
@given(st.lists(st_isin_codes()), st.lists(st.text()), st.randoms())
def check_property_extract_isin_codes_2(isins, tokens, random):
    
    tokens += isins
    random.shuffle(tokens)
    text = ' '.join(tokens)
    
    actual = sorted(extract_isin_codes(text))
    expected = sorted(isins)
    
    assert actual == expected 

In [ ]:
!sh pytest.sh check_property_extract_isin_codes_2

In [ ]:
class TextWithIsins:
    
    def __init__(self, isins, tokens, random):
        self.isins, self.tokens = isins, isins + tokens
        random.shuffle(self.tokens)
        
    def __repr__(self):
        return ' '.join(self.tokens)

    
st_text_with_isins = st.builds(TextWithIsins, 
    isins=st.lists(st_isin_codes()),
    tokens=st.lists(st.text()),
    random=st.randoms(),
)

In [ ]:
@settings(verbosity=Verbosity.verbose)
@given(st_text_with_isins)
def check_property_extract_isin_codes_3(text_with_isins):
    actual = sorted(extract_isin_codes(repr(text_with_isins)))
    expected = sorted(text_with_isins.isins)
    assert actual == expected 

In [ ]:
!sh pytest.sh check_property_extract_isin_codes_3 -s

## Stateful Testing

In [ ]:
class Queue(object):
    """FIFO queue with a maximum size"""

    def __init__(self, max_size):
        self._buffer = [None] * max_size
        self._in, self._out, self.max_size = 0, 0, max_size

    def put(self, item):
        self._buffer[self._in] = item
        self._in = (self._in + 1) % self.max_size

    def get(self):
        result = self._buffer[self._out]
        self._out = (self._out + 1) % self.max_size
        return result

    def __len__(self):
        return (self._in - self._out) % self.max_size

In [ ]:
import itertools

In [ ]:
operations = 'new', 'put', 'get', 'size'
list(itertools.permutations(operations))

In [ ]:
from hypothesis.stateful import RuleBasedStateMachine
from hypothesis.stateful import rule, precondition

In [ ]:
class QueueStateMachine(RuleBasedStateMachine):
    
    Actual, Model = Queue, list

    is_created = lambda self: hasattr(self, 'model')
    is_not_created = lambda self: not hasattr(self, 'model')
    is_not_empty = lambda self: hasattr(self, 'model') \
                                and self.model
    
    @precondition(is_not_created)
    @rule(max_size=st.integers(min_value=1, max_value=10))
    def new(self, max_size):
        self.actual = self.Actual(max_size) 
        self.model = self.Model()
        self.max_size = max_size

    @precondition(is_created)
    @rule(item=st.integers())
    def put(self, item):
        self.actual.put(item), self.model.append(item)

    @precondition(is_not_empty)
    @rule()
    def get(self):
        actual, model = self.actual.get(), self.model.pop()
        assert actual == model

    @precondition(is_created)
    @rule()
    def length(self):
        actual, model = len(self.actual), len(self.model)
        assert actual == model

In [ ]:
check_property_queue_matches_model_1 = QueueStateMachine.TestCase

In [ ]:
class QueueStateMachine(QueueStateMachine):
    # this is a total cheat, the order in which the errors would naturally occur is 
    # system under test, model, specification. However the narrative is a little
    # better when presented as model, specification, system under test, so secretly
    # fix the bug with the system under test here
    class Actual(Queue):
        def __init__(self, max_size):
            super().__init__(max_size + 1)
            
check_property_queue_matches_model_1 = QueueStateMachine.TestCase

In [ ]:
!sh pytest.sh check_property_queue_matches_model_1

In [ ]:
class QueueStateMachine2(QueueStateMachine):
        
    is_not_full = lambda self: (hasattr(self, 'model')
                                and len(self.model) < self.max_size)

    @precondition(is_not_full)
    @rule(item=st.integers())
    def put(self, item):
        super().put(item)
        
check_property_queue_matches_model_2 = QueueStateMachine2.TestCase

In [ ]:
!sh pytest.sh check_property_queue_matches_model_2

In [ ]:
class QueueStateMachine2(QueueStateMachine2):
    # undo the cheating fix
    Actual = Queue

In [ ]:
class QueueStateMachine3(QueueStateMachine2):
    
    @precondition(QueueStateMachine2.is_not_full)
    @rule(item=st.integers())
    def put(self, item):
        self.actual.put(item), self.model.insert(0, item)
        
check_property_queue_matches_model_3 = QueueStateMachine3.TestCase

In [ ]:
!sh pytest.sh check_property_queue_matches_model_3

In [ ]:
queue = Queue(max_size=1)
queue._in, queue._out

In [ ]:
queue.put(0)
queue._in, queue._out

In [ ]:
def put(self, item):
    self._buffer[self._in] = item
    self._in = (self._in + 1) % self.max_size

In [ ]:
class Queue2(Queue):
    def __init__(self, max_size):
        super(Queue2, self).__init__(max_size + 1)

class QueueStateMachine4(QueueStateMachine3):
    Actual = Queue2
    
check_property_queue_matches_model_4 = QueueStateMachine4.TestCase

In [ ]:
!sh pytest.sh check_property_queue_matches_model_4

&nbsp;

* Automatic testing of stateful systems

* Interpreting Errors

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;• Specification

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;• Model

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;• System under test

&nbsp;

# Stop Writing Tests...

# ...Start Generating Them!

# Shrinking - Extract signal from noise

# Look for property patterns

# Stateful testing - Auto integration tests

In [79]:
%%javascript
$('#clear_all_output').click()

<IPython.core.display.Javascript object>

In [80]:
%%HTML
<link href="https://fonts.googleapis.com/css?family=ABeeZee" rel="stylesheet">
<style>body { font-family: 'ABeeZee', serif !important; }</style>